In [ ]:
import matplotlib.pyplot as plt
figure_path = '/home/thoresen/Code/deep_learning/02456-2021-project/figures/'
import fpmodules as fp
from fpmodules import BlobManager
import os
from tools.constants import *
from fpmodules.fpio.raw_data import load_data_from_files
from tools.evex_scout import *
from tools.evex_calc import *
from tools.generate_data import *
from tools.load_data import *
import datetime
from tools.generate_data import create_labelled_data

In [ ]:
chunks = False
mac = 'dca6327d8fa8'

measurements = fp.dbquery('select * from measurement where sessionid=686')
dates = measurements['DateId'].sort_values().unique().tolist()
delete_files = False

date = dates[0]
blob_mgr = BlobManager(configuration='rclone')
blob_list = blob_mgr.list_blobs(container='scouts', subdir=f"{mac}/{str(date)}/raw/")
blob_list = [b for b in blob_list if b.endswith(".raw.gz")]
blob = blob_list[78]
print('Number of raw data files:', len(blob_list))
file_list = blob_mgr.download_blobs([blob], RAWDATA_CACHE_PATH, container='scouts')
data, times = load_data_from_files(file_list, RAWDATA_CACHE_PATH)

raw_start = pd.Timestamp(datetime.datetime.utcfromtimestamp(int(times[0])))
raw_end = pd.Timestamp(datetime.datetime.utcfromtimestamp(int(times[-1])))
measurements, insects = get_insects_and_measurements(0)

in_range = (measurements['Datetime'] >= raw_start) & (measurements['Datetime'] <= raw_end)
_meas = measurements[in_range]
_meas = _meas.rename(columns={'Id': 'MeasurementId'})
_meas = _meas.sort_values('MeasurementId')

In [ ]:
start_inds, stop_inds = get_start_and_stop(data, times)
tight_start = start_inds.copy()
tight_end = stop_inds.copy()

In [ ]:
for i in range(0, len(start_inds)):
    tight_start[i], tight_end[i] = tight_cut(data, start_inds[i], stop_inds[i])

In [ ]:
plt.figure(figsize=(30,5))
plt.plot(data[0:900000,7])
for s in start_inds:
    if s < 900000:
        plt.axvline(s, c='C1')
for e in stop_inds:
    if e < 900000:
        plt.axvline(e, c='C2')
plt.show()

In [ ]:
plt.figure(figsize=(30,5))
plt.plot(data[0:900000,7])
for s in tight_start:
    if s < 900000:
        plt.axvline(s, c='C1')
for e in tight_end:
    if e < 900000:
        plt.axvline(e, c='C2')
plt.show()

In [ ]:
_meas = _meas.sort_values(['Datetime','MeasurementId'])


In [ ]:
channel=7
segment=4
labels = np.zeros_like(data)
for i, dt in enumerate(_meas['Datetime'].tolist()[0:11]):
    insect_measurement = insects[(insects['Datetime'] == dt) & (insects['SegmentId'] == segment)]

    if len(insect_measurement['MeasurementId'].unique()) > 1:
        print('found double')
        print(dt)
    #if channel in insect_measurement['SegmentId'].tolist():
        
    #    labels[tight_start[i]:tight_end[i], 7] = 1
        #ev = fp.Event(m_id).fill().plot()
        #plt.show()

In [ ]:
channel=4

labels = np.zeros_like(data)
for i, m_id in enumerate(_meas['MeasurementId'].tolist()[0:11]):
    insect_measurement = insects[insects['MeasurementId'] == m_id]

    if channel in insect_measurement['SegmentId'].tolist():
        labels[tight_start[i]:tight_end[i], 7] = 1
        ev = fp.Event(m_id).fill().plot()
        plt.show()

In [ ]:
plt.figure(figsize=(30,5))
limit = 900000
plt.plot(data[0:limit,7],label='7')
plt.plot(labels[0:limit,7]*10000)
plt.legend()
plt.show()

In [ ]:
for m in _meas['Id'].tolist()[0:11]:#35421179
    ev = fp.Event(m).fill().plot()
    plt.show()

In [ ]:
f = 'dca6327d8fa8_20210330_raw_20210330T160022Z'
file = '/home/frth/EventCache/RawLabelledData/'+f+'_ds_10_data.npy'
file_label = '/home/frth/EventCache/RawLabelledData/'+f+'_ds_10_labels.npy'

In [ ]:
data_channel = np.load(file)
label_channel = np.load(file_label)

In [ ]:
label_channel.dtype

In [ ]:
plt.figure(figsize=(30,5))

plt.plot(data_channel[0:(90000)])
plt.plot(label_channel[0:(90000)]*0.001)
plt.show()

In [ ]:
for i in range(0,len(data_channel),50000):
    plt.figure(figsize=(30,5))

    plt.plot(data_channel[i:(i+50000)])
    plt.plot(label_channel[i:(i+50000)]*0.001)
    plt.show()

In [ ]:
plt.figure(figsize=(30,5))

plt.plot(data_channel[(i+10000):(i+20000)])
plt.plot(label_channel[(i+10000):(i+20000)]*0.001)
plt.gca().set_ylim([0.00026,0.0003])
plt.show()